In [2]:
import pickle
import pandas as pd
import numpy as np
import psycopg2 as pg

In [3]:
def web_ent_to_label(web_ent_list):
    if type(web_ent_list) is not list:
        if type(web_ent_list) == float:
            return ''
        if web_ent_list is None:
            return ''
    web_label = ""
    for entity in web_ent_list:
        if 'description' in entity.keys():
            web_label += entity['description'] + ' '

    return web_label

In [4]:
def img_match_to_keywords(img_match_list):
    if type(img_match_list) is not list:
        if type(img_match_list) == float:
            return ''
        if img_match_list is None:
            return ''
    keywords = ""
    for match in img_match_list:
        url = match['url']
        if url[-1] == '/':
            url = url[:-1]
        kws = url.split('/')[-1]
        h_kws = kws.count('-')
        u_kws = kws.count('_')
        if u_kws > h_kws:
            spl = kws.split('_')
            for unigram in spl:
                try:
                    int(unigram)
                except:
                    if len(unigram) < 12:
                        keywords += unigram + ' '
        elif u_kws == 0 and h_kws == 0:
            continue
        elif  h_kws >= u_kws:
            spl = kws.split('-')
            for unigram in spl:
                try:
                    int(unigram)
                except:
                    keywords += unigram + ' '
    return keywords

In [5]:
def img_match_to_num(img_match_list):
    if type(img_match_list) is not list:
        return 0
    return len(img_match_list)

In [6]:
def get_data(table_name='group_members'):
    db = pg.connect(dbname='postgres', host='35.196.103.17',
                    user='postgres', password='meaty')
    cur = db.cursor()
    query = "SELECT * FROM " + table_name + ";"
    cur.execute(query)
    if table_name == 'group_members':
        col_names = ['affil', 'fb_name',
                     'date_added', 'school', 'approver_name']
    else:
        col_names = ['id', 'school', 'poster_name', 'post_time', 'title', 'caption', 'price',
                     'num_reacts', 'angrys', 'hahas', 'likes', 'loves', 'prides', 'sads',
                     'thankfuls', 'wows', 'reacts_url', 'url', 'post_date', 'post_hour',
                     'img_hash', 'school_name_id']
    df = pd.DataFrame(cur.fetchall(), columns=col_names)
    return df

In [17]:
post_df = get_data('posts')
post_df.set_index('id', inplace=True)
for col in ['title', 'caption']:
    post_df[col].loc[post_df[col].isnull()] = ''


c:\program files\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
res = pickle.load(open('api-batches.pkl', 'rb'))

In [18]:
df = pd.DataFrame(res).T
df.index.rename('post_id', inplace=True)
df.text = df.text.str.replace('\n', ' ')

In [19]:
df['web_labels'] = df['web_entities'].apply(web_ent_to_label)
df.drop('web_entities', axis=1, inplace=True)

In [20]:
df['num_similar_imgs'] = df['img_match'].apply(img_match_to_num)
df['url_keywords'] = df['img_match'].apply(img_match_to_keywords)
df.drop('img_match', axis=1, inplace=True)

In [21]:
df.columns = ['v_' + gvis_column for gvis_column in df.columns]

In [22]:
df.v_num_similar_imgs[df.v_num_similar_imgs.isnull()] = 0
df.v_num_similar_imgs = df.v_num_similar_imgs.astype(int)

for col in ['v_text', 'v_labels', 'v_logos', 'v_web_labels']:
    df[col].loc[df[col].isnull()] = ''
df.to_csv('vision-all-fmtd.csv', encoding='utf-8')

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\program files\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
all_data_joined = post_df.join(df)

In [30]:
all_data_joined['all'] = all_data_joined['title'] + ' ' + \
    all_data_joined['caption'] + ' ' + all_data_joined['v_text'] + ' ' + all_data_joined['v_labels'] + \
    ' ' + all_data_joined['v_logos'] + ' ' + all_data_joined['v_web_labels']
all_data_joined['all'] = all_data_joined['all'].str.lower()

In [31]:
q_df = all_data_joined[all_data_joined['all'].str.contains('stress', na=False)]
q_counts = q_df.groupby('school').count()['caption']
q_counts

school
brown         26
columbia     103
cornell       30
dartmouth      4
harvard        8
penn          25
princeton     16
yale          11
Name: caption, dtype: int64

In [33]:
q_df = all_data_joined[all_data_joined['all'].str.contains('harvard', na=Falselse)]
q_counts = q_df.groupby('school').count()['caption']
q_counts

school
brown         17
columbia      36
cornell       91
dartmouth      5
harvard      587
penn          27
princeton     37
yale          75
Name: caption, dtype: int64